In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')

Define tools

In [6]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL

tavily_tool = TavilySearchResults(max_results=3)

repl = PythonREPL()

@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."]
):
    """Use this to execute python code. If you wnat to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user./"""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    
    result_Str = f"Successfully executed: \n```python\n{code}\n```\nStdout: {result}"
    return (
        result_Str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER"
    )

Create graph

In [7]:
def make_system_prompt(suffix: str) -> str:
    return(
        "You are a helpful AI assistant, collabrating with other assistants."
        " Use the provided tools to progress towards answering the question. "
        "If yuo are unable to fully answer, that's OK, another assistant with different tools "
        "will help where you left off. Execute what you can to make progress. "
        "If you or any of the other assistants have the final answer or deliverable, "
        "prefix your response with FINAL ANSWER so the team knows to stop."
        f"\n{suffix}"
    )

In [ ]:
from typing import Literal

from langchain_core.messages import BaseMessage, HumanMessage
from langchain_groq import ChatGroq
from langgraph.prebuilt import create_react_agent
from langgraph.graph import MessagesState, END
from langgraph.types import Command

llm = ChatGroq(model="Gemma2-9b-It")

def get_next_node(last_message: BaseMessage, goto: str):
    if 'FINAL ANSWER' in last_message.content:
        return END
    
    return goto

research_agent = create_react_agent(
    llm,
    tools = [tavily_tool],
    prompt=make_system_prompt(
        "You can only do research. You are working with a chart generator colleague."
    ),
)

def